# 2 Thread

In Chapter 1, we have seen how to use Process to accelerate your python program. In this chapter we will see:

1. What threads are
2. How to create threads and wait for them to finish
3. How to use a ThreadPoolExecutor
4. How to avoid race conditions
5. How to use the common tools that Python threading provide

## 2.1 What Is a Python Thread?

Generally speaking, a thread is a separate flow of execution, which means that your program will have many flow of execution happening at the same time. But for most Python 3 implementations **the threads do not actually execute at the same time**.

This is due to the python interpreter uses [GIL(Global Interpreter Lock)](https://realpython.com/python-gil/) to provide a thread safe memory management environment for the underlying C libraries. And **GIL limits only one Python thread can run at a time**.

## 2.2 When to use python thread?

As only one python thread can run at a time, if **tasks that spend much of their time waiting for external events are generally good candidates for threading**.

If **tasks that require heavy CPU computation and spend little time waiting for external events might not run faster at all after adding multi threading**.

**The above condition is true for code written in Python and running on the standard CPython implementation**. If your threads are written in C they have the ability to release the GIL and run concurrently. If you are running on a different Python interpreter implementation, check with the documentation to see how it handles threads.

**If you are running a standard Python implementation, writing in only Python, and have a CPU-bound problem, you should check out the multiprocessing module instead.**

So Why python still have thread, if they can't run parallely?
Thread can provide gains in design clarity for your program architecture. Most of the examples you’ll see in this chapter are not necessarily going to run faster because they use threads. Using threading in them helps to make the design cleaner and easier to reason about.

## 2.3 A simple thread

The Python standard library provides threading, which contains most of the primitives you’ll see in this article. Thread, in this module, nicely encapsulates threads, providing a clean interface to work with them.

To start a separate thread, you create a Thread instance and then tell it to .start():


In [1]:
import logging
import time
from threading import Thread

In [2]:
# define a thread task
def thread_task(name: str):
    logging.info(f"Thread {name} : start")
    time.sleep(2)
    logging.info(f"Thread {name} : stop")

In [5]:

## main process that launches thread
# set up logger

level="INFO"
date_format="%(asctime)s: %(message)s"
logging.basicConfig(level=level,datefmt=date_format)

# start the main process
logging.info("Main    : before creating thread")

# create thread
logging.info("Main    : before running thread-1")
thread1=Thread(target=thread_task,args=("thread-1",))
logging.info("Main    : before running thread-2")
thread2=Thread(target=thread_task,args=("thread-2",))

# start thread
thread1.start()
thread2.start()


logging.info("Main    : wait for the thread-1 to finish")
thread1.join()

logging.info("Main    : wait for the thread-2 to finish")
thread2.join()
# finish main process
logging.info("Main    : all done")



INFO:root:Main    : before creating thread
INFO:root:Main    : before running thread-1
INFO:root:Main    : before running thread-2
INFO:root:Thread thread-1 : start
INFO:root:Thread thread-2 : start
INFO:root:Main    : wait for the thread-1 to finish
INFO:root:Thread thread-1 : stop
INFO:root:Main    : wait for the thread-2 to finish
INFO:root:Thread thread-2 : stop
INFO:root:Main    : all done


In the above example, when you create a Thread, you pass it **a function and a list containing the arguments to that function**. In our case, you’re telling the Thread to run thread_task() and to pass it `thread-1` as an argument.

We recommend you to always give a custom thread name. There is **threading.get_ident()**, which returns a unique name for each thread, but these are usually neither short nor easily readable.


### The join() method

Try to comment the join(). You will notice the main finish before thread. Run src/SimpleThreadExample.py, it's more clear than the jupyter notebook.
So the join() method tells the main process to wait the thread that calls join().


### Daemon Threads

**In computer science, a daemon is a process that runs in the background.**

**In Python threading, a daemon thread will shut down immediately when the program exits.** As a result, you don't need to worry about shutting the daemon thread down.

If a program is running Threads that are not daemons, then the program will wait for those threads to complete before it terminates. Threads that are daemons, however, are just killed wherever they are when the program is exiting.

You can create a daemon thread by adding the **daemon=True** flag into the Thread constructor. Check src/DaemonThreadExample.py for example.

Run src/DaemonThreadExample.py, you will have below output

```text
14:24:03: Main    : before creating thread
14:24:03: Main    : before running thread-1
14:24:03: Main    : before running thread-2
14:24:03: Thread thread-1 : start
14:24:03: Thread thread-2 : start
14:24:03: Main    : wait for the thread-1 to finish
14:24:03: Main    : wait for the thread-2 to finish
14:24:03: Main    : all done
```

You can notice the two thread are started, by never finished. Because the main process finish before thread task, as the thread are daemon thread. They are automatically shut down after the main terminates. If you add the join, then the thread can finish normally.

## 2.4 Working with many threads

Similar to Process, when you have many Thread to create and stop, you need to use a **ThreadPoolExecutor, and it’s part of the standard library in concurrent.futures (as of Python 3.2)**. Otherwise, the thread context creation may take much more time than your task.

The ThreadPoolExecutor has three principal functions:
- submit(): Dispatch a function to be executed and return a future object.
- map(): Apply a function to an iterable of elements.
- shutdown(): Shut down the executor.

Below figure shows the life cycle of a thread pool executor

![python_thread_pool_life_cycle.PNG](../../images/python_thread_pool_life_cycle.PNG)
### Future object

A future is an object that **represents a delayed result for an asynchronous task**.

It is also sometimes called a `promise` or a `delay`. It provides a context for the result of a task that may or may not be executing and a way of getting a result once it is available.

In Python, the Future object is returned from an Executor, such as a ThreadPoolExecutor when calling the submit() function to dispatch a task to be executed asynchronously.

In general, we do not create Future objects; we only receive them and we may need to call functions on them.

The Future object provides a number of helpful functions for inspecting the status of the task such as: **cancelled(), running(), and done()** to determine if the task was cancelled, is currently running, or has finished execution.

- cancelled(): Returns True if the task was cancelled before being executed.
- running(): Returns True if the task is currently running.
- done(): Returns True if the task has completed or was cancelled.

### Simple thread pool executor example

The easiest way to create it is as a context manager, using the **with** statement to manage the creation and destruction of the pool. Once we have the executor, we can submit tasks to the pool via two below methods:
- map:
- submit:

Check src/ThreadPoolExample.py for a detailed example




## 2.5 Race condition

**Race conditions can occur when two or more threads access a shared piece of data or resource.** Frequently, they only occur rarely, and they can produce confusing results. As you can imagine, this makes them quite difficult to debug.

Check **src/RaceConditionExample.py**, and run it. You will find below output:
```text
18:49:51: Testing update. Starting value is 0.
18:49:51: Thread 0: starting update
18:49:51: Thread 0 has init db value 0
18:49:51: Thread 1: starting update
18:49:51: Thread 1 has init db value 0
18:49:51: Thread 1 output value 1 to db
18:49:51: Thread 1: finishing update
18:49:51: Thread 0 output value 1 to db
18:49:51: Thread 0: finishing update
18:49:51: Testing update. Ending value is 1.
```

Surprise, the ending value in database is not 2. Because thread 1, thread 2 start at same time, they both read init value 0. After calculation, they both have 1 as result. Then they both write to db with value 1. The order who writes first to db is not determined, but we know the latter will overwrite the former, not increment the result of the former. This is a typical **race condition**. Instead of coordinate the result between threads, thread overwrites the result of each other.

## 2.6 Basic Synchronization Using Lock

The easiest way to solve race conditions is to use a **Lock**, which allows only one thread at a time to access the read-modify-write shared resources (can be data or code).

Check **src/LockExample.py**, we use a lock to protect the reading and writing of the fake database. It means if a thread is access the database, other thread can't access the database.

Run it, you will have below output:

```text
19:18:50: Testing update. Starting value is 0.
19:18:50: Thread 0: starting update
19:18:50: Thread 0 acquired lock
19:18:50: Thread 0 has init db value 0
19:18:50: Thread 1: starting update
19:18:50: Thread 0 released lock
19:18:50: Thread 0 output value 1 to db
19:18:50: Thread 0: finishing update
19:18:50: Thread 1 acquired lock
19:18:50: Thread 1 has init db value 1
19:18:50: Thread 1 released lock
19:18:50: Thread 1 output value 2 to db
19:18:50: Thread 1: finishing update
19:18:50: Testing update. Ending value is 2.
```

You can notice the output is correct this time.

## 2.6.1 Dead lock

When we use lock incorrectly, we may create another issue **Deadlock**. Deadlocks usually happen from one of two subtle things:

- An implementation bug where a Lock is not released properly
- A design issue where a utility function needs to be called by functions that might or might not already have the Lock


The first situation happens sometimes, but **using a Lock as a context manager greatly reduces** how often. It is recommended to write code whenever possible to make use of context managers, as they help to avoid situations where an exception skips you over the .release() call.


The design issue can be a bit trickier in some languages. Thankfully, Python threading has a second object, called **RLock**, that is designed for just this situation. It allows a thread to .acquire() an RLock multiple times before it calls .release(). That thread is still required to call .release() the same number of times it called .acquire(), but it should be doing that anyway.

Lock and RLock are two of the basic tools used in threaded programming to prevent race conditions. There are a few other that work in different ways. Before you look at them, let’s shift to a slightly different problem domain.

For example, below code will create a deadlock condition.
```python
import threading

l = threading.Lock()
print("first acquire")
l.acquire()
print("second acquire")
l.acquire()
print("acquired lock twice")
```

For a complete example, please check **src/DeadlockExample**. Try to activate the second acquire and see what happens.

## 2.7 Producer-Consumer Threading

The Producer-Consumer Problem is a standard computer science problem used to look at threading or process synchronization issues.

In below example, we are going to simulate a program that needs to read messages from a network and write them to disk. The program does not request a message when it wants. It must be listening and accept messages as they come in. The messages will not come in at a regular pace, but will be coming in bursts. This part of the program is called the producer.

On the other side, once you have a message, you need to write it to a database. The database access is slow, but fast enough to keep up to the average pace of messages. It is not fast enough to keep up when a burst of messages comes in. This part is the consumer.

In between the producer and the consumer, you will create a Pipeline that will be the part that changes as you learn about different synchronization objects.

That’s the basic layout. Let’s look at a solution using Lock. It doesn’t work perfectly, but it uses tools you already know, so it’s a good place to start.

## 2.7.1 Producer-Consumer Using Lock

The general design is that there is a producer thread that reads from the fake network and puts the message into a Pipeline:

Check **src/ProducerConsumerLock.py** for complete example.

## 2.7.2 Producer-Consumer using queue
Python’s standard library has a **queue module which, in turn, has a Queue class**. Let’s change the Pipeline to use a Queue instead of just a variable protected by a Lock. You’ll also use a different way to stop the worker threads by using a different primitive from Python threading, an **Event**.


The **threading.Event object allows one thread to signal an event while many other threads can be waiting for that event to happen**. The key usage in this code is that the threads that are waiting for the event do not necessarily need to stop what they are doing, they can just check the status of the Event every once in a while.

Check **src/ProducerConsumerQueue.py** for complete example

## 2.8 Threading Objects
There are a few more primitives offered by the Python threading module. While you didn’t need these for the examples above, they can come in handy in different use cases, so it’s good to be familiar with them.

### 2.8.1 Semaphore
**A Semaphore(threading.Semaphore)** is a counter with a few special properties:
 - counting is atomic: This means that there is a guarantee that the operating system will not swap out the thread in the middle of incrementing or decrementing the counter.
 - The internal counter is incremented when you call .release() and decremented when you call .acquire().
 - If a thread calls .acquire() when the counter is zero, that thread will block until a different thread calls .release() and increments the counter to one.

**Semaphores are frequently used to protect a resource that has a limited capacity**. An example would be if you have a pool of connections and want to limit the size of that pool to a specific number.



### 2.8.2 Timer

**A threading.Timer is a way to schedule a function to be called after a certain amount of time has passed.** You create a Timer by passing in a number of seconds to wait and a function to call:

```python
t = threading.Timer(30.0, my_function)
```

You start the Timer by calling .start(). The function will be called on a new thread at some point after the specified time, but be aware that there is no promise that it will be called exactly at the time you want.

If you want to stop a Timer that you’ve already started, you can cancel it by calling .cancel(). **Calling .cancel() after the Timer has triggered does nothing and does not produce an exception.**

**A Timer can be used to prompt a user for action after a specific amount of time. If the user does the action before the Timer expires, .cancel() can be called.**

### 2.8.3 Barrier

**A threading.Barrier can be used to keep a fixed number of threads in sync.** When creating a Barrier, the caller must specify how many threads will be synchronizing on it. Each thread calls .wait() on the Barrier. They all will remain blocked until the specified number of threads are waiting, and then the are all released at the same time.

Remember that threads are scheduled by the operating system so, even though all of the threads are released simultaneously, they will be scheduled to run one at a time.

One use for a Barrier is to allow a pool of threads to initialize themselves. Having the threads wait on a Barrier after they are initialized will ensure that none of the threads start running before all of the threads are finished with their initialization.